{ “cells”: \[ { “cell_type”: “code”, “execution_count”: null,
“metadata”: {}, “outputs”: \[\], “source”: \[ “import importlib.util”,
“import sys”, “import torch”, “import math”, “import os”, “import pandas
as pd”, “import numpy as np”, “from tqdm import tqdm”, “from
nltk.translate.bleu_score import sentence_bleu, SmoothingFunction”,
“from datasets import load_dataset, Dataset, concatenate_datasets”,
“from sklearn.model_selection import train_test_split”, “import psutil”,
“import gc”, “import time”, “import requests”, “”, “\# Check if
rouge_score is installed”, “if importlib.util.find_spec("rouge_score")
is None:”, ” print("Error: ‘rouge_score’ module not found. Please
install it manually in a virtual environment or using pipx:")“,”
print("1. Create a virtual environment: python3 -m venv venv")“,”
print("2. Activate it: source venv/bin/activate")“,” print("3. Install:
pip install rouge_score")“,” print("Or use pipx: pipx install
rouge_score")“,” sys.exit(1)“,”“,”from rouge_score import
rouge_scorer“,”from transformers import AutoModelForCausalLM,
AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer,
Trainer, TrainingArguments, DataCollatorForLanguageModeling,
pipeline“,”“,”\# Load and merge datasets“,”def
load_and_merge_datasets():“,” \# STEP 1: Load HOPE Therapy Data“,”
hope_path =
"/content/SPARTA_WSDM2022/HOPE_data/HOPE_therapy_session_transcripts"“,”
try:“,” files = \[f for f in os.listdir(hope_path) if
f.endswith(".csv")\]“,” except FileNotFoundError:“,” print(f"Error: HOPE
dataset directory not found at {hope_path}")“,” files = \[\]“,”“,”
hope_pairs = \[\]“,” for file in files:“,” df =
pd.read_csv(os.path.join(hope_path, file))“,” \# Map the Type column to
proper roles“,” df\[‘Speaker’\] = df\[‘Type’\].map({‘T’: ‘Therapist’,
‘P’: ‘Client’})“,” df\[‘Content’\] = df\[‘Utterance’\] \# Rename for
consistency“,”“,” \# Extract consecutive client-therapist exchanges“,”
for i in range(1, len(df)):“,” if df.loc\[i-1, ‘Speaker’\] == "Client"
and df.loc\[i, ‘Speaker’\] == "Therapist":“,” hope_pairs.append({“,”
"prompt": f"Client: {df.loc\[i-1, ‘Content’\]}",“,” "response":
f"Therapist: {df.loc\[i, ‘Content’\]}",“,” "source": "HOPE"“,” })“,”“,”
print(f"Extracted {len(hope_pairs)} dialogue pairs from HOPE
dataset")“,”“,” \# STEP 2: Load EmpatheticDialogues“,” try:“,”
empathetic_ds = load_dataset("empathetic_dialogues")“,” empathy_pairs =
\[\]“,”“,” \# Process conversations to get contextual exchanges“,”
prev_conv_id = None“,” context = ""“,”“,” for row in
empathetic_ds\[‘train’\]:“,” if row\[‘utterance_idx’\] \> 0 and
row\[‘conv_id’\] == prev_conv_id:“,” empathy_pairs.append({“,” "prompt":
f"Client: {context}",“,” "response": f"Therapist:
{row\[‘utterance’\]}",“,” "emotion": row\[‘context’\],“,” "source":
"EmpatheticDialogues"“,” })“,”“,” context = row\[‘utterance’\]“,”
prev_conv_id = row\[‘conv_id’\]“,”“,” print(f"Extracted
{len(empathy_pairs)} dialogue pairs from EmpatheticDialogues
dataset")“,” except Exception as e:“,” print(f"Error loading
EmpatheticDialogues: {e}")“,” empathy_pairs = \[\]“,”“,” \# STEP 3: Load
CounselChat“,” try:“,” \# Download CounselChat data“,” url =
"https://raw.githubusercontent.com/nbertagnolli/counsel-chat/master/data/counselchat-data.csv"“,”
response = requests.get(url)“,” with open("counselchat-data.csv", "wb")
as f:“,” f.write(response.content)“,”“,” cc_df =
pd.read_csv("counselchat-data.csv")“,” counsel_pairs = \[\]“,”“,” for
\_, row in cc_df.iterrows():“,” if pd.notnull(row\[‘questionText’\]) and
pd.notnull(row\[‘answerText’\]):“,” counsel_pairs.append({“,” "prompt":
f"Client: {row\[‘questionText’\]}",“,” "response": f"Therapist:
{row\[‘answerText’\].replace(’\] } \], “metadata”: { “kernelspec”: {
“display_name”: “Python 3”, “language”: “python”, “name”: “python3” },
“language_info”: { “codemirror_mode”: { “name”: “ipython”, “version”: 3
}, “file_extension”: “.py”, “mimetype”: “text/x-python”, “name”:
“python”, “nbconvert_exporter”: “python”, “pygments_lexer”: “ipython3”,
“version”: “3.12.0” } }, “nbformat”: 4, “nbformat_minor”: 5 }